## Classical LA methods applied to the ASCAD database
###### This includes my implementations of $\chi^2$ and Welch's t-test.

In [111]:
from settings.ascad import *

In [112]:
import h5py

ascad_hdf = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_VAR}/{ASCADDataType.default}.h5", 'r')

keys = list(ascad_hdf.keys())
keys

['Attack_traces', 'Profiling_traces']

In [113]:
att_group = ascad_hdf["Attack_traces"]

att_keys = list(att_group.keys())
att_keys

['labels', 'metadata', 'traces']

In [114]:
att_traces = att_group["traces"]

att_traces.shape

(100000, 1400)

In [115]:
att_labels = att_group["labels"]

att_labels.shape

(100000,)

In [116]:
import tqdm

for _ in tqdm.tqdm(att_traces[1:10]):
    pass

100%|██████████| 9/9 [00:00<00:00, 31562.49it/s]


In [117]:
att_head = att_traces[1:10]

In [118]:
from settings.nbloader import NotebookLoader

nb = NotebookLoader("../").load_module("tvla.welch_t_test")
tp = nb.TraceProcessor()

importing Jupyter notebook from ./welch_t_test.ipynb
(0.0, 598.0, 1.0) (0.0, 99, nan)
(0.0, 300.119807070072, 1.0) (-443.95712826391065, 99, nan)
False True


In [119]:
import numpy as np

fixed_1_ix = np.where(np.array(att_labels) == 1)[0]
fixed_2_ix = np.where(np.array(att_labels) == 2)[0]
random_not_1_ix = np.array(np.where(np.array(att_labels) != 1)[0])

fixed_1_ix.shape

(385,)

In [120]:
fixed_test_size = 10

fixed_1 = np.array(att_traces[fixed_1_ix])
fixed_2 = np.array(att_traces[fixed_2_ix])

len_fixed_1 = fixed_1.shape[0]
fixed_1a = np.array(att_traces[fixed_1_ix][:round(len_fixed_1/2)])
fixed_1b = np.array(att_traces[fixed_1_ix][round(len_fixed_1/2):])

random_not_1 = np.array(att_traces[random_not_1_ix[:len_fixed_1]])

In [121]:
all_ixs = range(att_traces.shape[0])

random_a_ix = sorted(np.random.choice(all_ixs, len_fixed_1, replace=False))
random_b_ix = sorted(np.random.choice(all_ixs, len_fixed_1, replace=False))

semi_random_b_ix = sorted(np.random.choice(list(set(all_ixs).difference(set(random_a_ix))), len_fixed_1, replace=False))

In [122]:
random_a = np.array(att_traces[random_a_ix])
random_b = np.array(att_traces[random_b_ix])
semi_random_b = np.array(att_traces[np.sort(semi_random_b_ix)])

# Classical LA methods
## $t$-test method

In [123]:
def nonzero_bins(bins):
    """
    Retrieves all bins for which at least one category has a non-zero value.

    :param bins: a set of categories with bins corresponding to that category.
    :return: the indexes of non-zero bins.
    """
    nz_bins = []
    for ix, a, b in zip(range(len(bins[0])), *bins):
        if a == 0 and b == 0:
            continue

        nz_bins.append(ix)

    return nz_bins

In [124]:
from collections import Counter

DEFAULT_INDEX = range(-128, 127)

def extract_ctable(traces):
    """
    Builds a contingency table from traces from the ASCAD dataset.

    :param traces: the traces from which the contingency table should be constructed.
    :return: the contingency table.
    """
    df = pd.DataFrame([Counter(bins) for bins in traces])
    res = df.sum().sort_index().reindex(DEFAULT_INDEX, fill_value=0).values

    return np.array(res, dtype=int)

In [ ]:
def starts_with(first, last, reverse=False):
    last += 1

    ls = range(first, last)
    if reverse:
        reversed(ls)

    return dict(zip(ls, [0] * len(list(ls))))

def sample_mean(trace: np.array):
    return trace.sum() / len(trace)


In [ ]:
def values(trace):
    return np.mean(trace), np.var(trace), len(trace)

def test_statistic(mean_0, var_0, car_0, mean_1, var_1, car_1):
    return (mean_0 - mean_1) / math.sqrt(var_0 / car_0 + var_1 / car_1)

def degree_of_freedom(var_0, car_0, var_1, car_1):
    nom = (var_0 / car_0 + var_1 / car_1)**2
    den = (var_0 / car_0)**2 / (car_0 - 1) + (var_1 / car_1)**2 / (car_1 - 1)

    return nom / den

In [ ]:
# Student's t-probability distribution function
import scipy.stats as stats

def cdf(t, v):
    """
    The cumulative distribution function as described on page 3.

    The aim of a t-test is to provide a quantitative value as a probability that the mean μ
        of two sets are different.

    :param t: the test statistic of the two traces.
    :param v: the degree of freedom of the two traces.
    :return: the cumulative distribution function.
    """
    return 2 * stats.t(v).cdf(-abs(t))

In [ ]:
import scipy.special as special

class TraceProcessor:
    def __init__(self, order=1, prob=.01):
        """
        TraceProcessor digests traces using incremental programming. This allows for t-testing
        anywhere in the digestion process.

        :param order: The order of t-test to be performed on the data. Defaults to 1 to accommodate
            for the ASCAD dataset, which features first-order leakages in AES traces.
        :param prob: The desired probability for the t-test to reject H0.
        """
        self.order = order
        self.prob = prob

        # Maximal statistical order to be maintained
        self.max_d = self.order + 1

        # Store binomial coefficients for use in calculating the central sums
        #   (Schneider & Moradi, 2016 - page 6 - formula 3).
        self.binom = {}
        for d in range(self.max_d):
            self.binom[d] = {}
            for k in range(self.max_d):
                self.binom[d][k] = special.binom(self.max_d, k)

        # Central Moments (CM) to be maintained.
        self.CM = starts_with(1, self.max_d)

        # Central Sums (CS) to be maintained, corresponding to the CMs.
        # CS is reversed to accommodate the update style of the CS (high to low).
        self.CS = starts_with(2, self.max_d, reverse=True)

        # Number of collected traces.
        self.n = 0
        # Cardinality of all observed traces combined.
        self.cardinality = 0

    def add_trace(self, trace: np.array):
        """
        Adds traces and computes intermediate central moments.

        :param trace: should be a numpy array containing the trace.
        """
        # Please note that n = cardinality Q' in all central moment calculations,
        #   where Q' is the trace set updated with the given trace.
        self.n += 1
        self.cardinality += len(trace)

        # Second half of Schneider & Moradi, 2016 - page 6 - formula 3 is incompatible with n == 1,
        #   due to the division by n - 1.
        if self.n <= 1:
            return

        # Assuming here that y is the sample mean of the new trace
        #   (Schneider & Moradi, 2016 - page 5 - bottom right)
        #   This is not specified in their paper, which merely states y as "trace".
        #   My assumption is based on the further calculations using delta as a number,
        #   where delta should be a vector of variable length without this assumption.
        delta = sample_mean(trace) - self.CM[1]

        for central_sum_order in self.CS:
            self.update_cs(central_sum_order, delta)

        for central_moment_order in self.CM:
            self.update_cm(central_moment_order, delta)

    def update_cm(self, d, delta):
        """
        Updates the Central Moment (CM) for a given order of central moment.

        :param d: Central Moment order.
        :param delta: the delta of the new trace (Schneider & Moradi, 2016 - page 5 - bottom left).
        """
        if d == 1:
            # Schneider & Moradi, 2016 - page 5 - bottom right.
            self.CM[1] += delta / self.n
        else:
            # Schneider & Moradi, 2016 - page 6 - top left.
            self.CM[d] = self.CS[d] / self.n

    def update_cs(self, d, delta):
        """
        Updates the Central Sum (CS) for a given order of central moment.

        :param d: Central Moment order.
        :param delta: the delta of the new trace (See page 5 bottom left).
        """
        sum_prev_cs = 0
        # Schneider & Moradi, 2016 - page 6 - formula 3.
        for k in range(2, d - 2):
            # As CS is reversed, this will access the lower-order central sums
            #   which are not yet updated with the new trace.
            sum_prev_cs += self.binom[d][k] * self.CS[d-k] * (-delta / self.n) ** k

        a = (((self.n - 1) / self.n) * delta) ** d
        b = 1 - (-1 / (self.n - 1)) ** (d - 1)

        self.CS[d] += sum_prev_cs + a * b

    def t_test(self, trace):
        """
        Returns whether to reject H0. H0 being "left and right are from different distributions".
        The traces should be normally distributed for the Student's t-test to work.

        :param trace: the trace under test.
        :return: whether H0 can be rejected.
        """
        # Note that the first-order Central Moment (CM[1]) corresponds to the mean and
        #   the second-order (CM[2]) to the variance.
        mean, var, car = values(trace)

        t = test_statistic(self.CM[1], self.CM[2], self.cardinality, mean, var, car)
        v = degree_of_freedom(self.CM[2], self.cardinality, var, car)

        # if t > 4.5 and v > 1000:
        #     # Schneider & Moradi, 2016 - page 3 - bottom left.
        #     return True

        return cdf(t, v)

tp = TraceProcessor()
for trc in fixed_1:
    tp.add_trace(trc)

num_trc = 100

sp_1 = 0
for trc in fixed_1[:num_trc]:
    sp_1 += tp.t_test(trc)

sp_2 = 0
for trc in fixed_2[:num_trc]:
    sp_2 += tp.t_test(trc)

print(sp_1 / num_trc)
print(sp_2 / num_trc)

In [125]:
import numpy as np
import math

def calc_t(H):
    mean, var, n = [[0, 0]] * 3
    nz_bins = nonzero_bins(H)

    for ix_cat in range(2):
        for ix_bin in nz_bins:
            item = H[ix_cat][ix_bin]
            # mean[ix_cat] += H[ix_cat][ix_bin] * range_h[ix_bin]
            mean[ix_cat] += item
            n[ix_cat] += item

        mean[ix_cat] /= n[ix_cat]

        for ix_bin in nz_bins:
            # tmp = (range_h[ix_bin] - mean[ix_cat])
            tmp = mean[ix_cat]
            # print(tmp)
            var[ix_cat] += tmp ** 2 + H[ix_cat][ix_bin]

        var[ix_cat] /= n[ix_cat]

    # t-value
    mean_diff = mean[0] - mean[1]
    var_sum = (var[0] / n[0]) + (var[1] / n[1])

    print(var_sum)
    return
    t_ret = mean_diff / math.sqrt(var_sum)

    # degree of freedom
    denom = ((var[0] / n[0]) * (var[0] / n[0])) / (n[0] - 1) + ((var[1] / n[1]) * (var[1] / n[1])) / (n[1] - 1)
    t_dof_ret = var_sum ** 2 / denom

    # cdf
    t_p_ret = 2 * stats.t(t_dof_ret).cdf(-abs(t_ret))

    return t_ret, t_dof_ret, t_p_ret

# TODO do not use the ctable!

calc_t(fixed_1[:2])

In [ ]:
calc_t([extract_ctable(fixed_1), extract_ctable(fixed_1)])

## $\chi^2$ method

In [127]:
import scipy.stats as stats

def calc_chi(ctable):
    """
    Calculates the p value for rejecting H0, among others.
    Small p values give evidence to reject the null hypothesis and conclude that for the
    scenarios presented in ctable the occurrences of the observations are not independent.

    :param ctable: contingency table for different categories of traces.
    :return: A 3-tuple containing: The value for chi, the degrees of freedom,
        the p value for rejecting H0.
    """
    num_cats = len(ctable)
    num_bins = len(ctable[0])

    # chi**2 value
    sum_rows = [0] * num_cats
    sum_cols = [0] * num_bins
    N = 0.0

    # Only check non-zero bins
    nz_bins = nonzero_bins(ctable)

    for ix_bin in nz_bins:
        for ix_cat in range(num_cats):
            # Bin from the contingency table
            c_bin = ctable[ix_cat][ix_bin]

            sum_rows[ix_cat] += c_bin
            sum_cols[ix_bin] += c_bin
            N += c_bin

    chi = 0.0
    for ix_bin in nz_bins:
        for ix_cat in range(num_cats):
            E = (sum_rows[ix_cat] * sum_cols[ix_bin]) / N
            tmp = (ctable[ix_cat][ix_bin] - E)

            chi += tmp ** 2 / E

    # Degrees of freedom
    dof = (num_bins - 1) * (num_cats - 1)
    # p-value for rejecting H0
    p = stats.chi2(dof).cdf(chi)

    return chi, dof, p

In [128]:
def p_value_chi(a, b):
    print(f"p-value: {calc_chi([extract_ctable(a), extract_ctable(b)])[2]:.3f}")

In [129]:
fixed_1

array([[-23, -26, -29, ..., -84, -86, -85],
       [-21, -25, -29, ..., -85, -84, -86],
       [-20, -24, -29, ..., -86, -85, -86],
       ...,
       [-21, -21, -25, ..., -84, -84, -85],
       [-20, -26, -29, ..., -84, -85, -85],
       [-21, -21, -26, ..., -85, -85, -85]], dtype=int8)

In [130]:
random_b

array([[-21, -25, -29, ..., -85, -86, -86],
       [-20, -21, -25, ..., -85, -85, -86],
       [-22, -25, -28, ..., -84, -85, -86],
       ...,
       [-20, -23, -29, ..., -85, -87, -86],
       [-20, -23, -29, ..., -85, -86, -86],
       [-22, -23, -26, ..., -84, -85, -84]], dtype=int8)

In [131]:
cases = {
    "Fixed vs. fixed, equal traces.": (fixed_1, fixed_1),
    "Fixed vs. fixed, equal key.": (fixed_1a, fixed_1b),
    "Fixed vs. fixed, different key.": (fixed_1, fixed_2),
    "Fixed vs. semi-random (fixed key is not in random sample).": (fixed_1, random_not_1),
    "Fixed vs. random.": (fixed_1, random_b),
    "Random vs. random.": (random_a, random_b),
    "Random vs. semi-random": (random_a, semi_random_b)
}

In [132]:
cases_chi = {
    "Scenario": list(cases.keys()),
    "p-value": [f"{calc_chi([extract_ctable(a), extract_ctable(b)])[2]:.3f}" for a, b in cases.values()]
    }

### Examples for $\chi^2$ on different scenarios.
Small p-values give reason to reject $H_0$ =
"the occurrences of these observations are independent".

In [133]:
pd.DataFrame(cases_chi)

,Scenario,p-value
0,"Fixed vs. fixed, equal traces.",0.000
1,"Fixed vs. fixed, equal key.",0.204
2,"Fixed vs. fixed, different key.",0.997
3,Fixed vs. semi-random (fixed key is not in ran...,0.120
4,Fixed vs. random.,1.000
5,Random vs. random.,0.108
6,Random vs. semi-random,1.000
